In [25]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import csv

df = pd.read_csv('data_geocoded.csv')
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,Unnamed: 0,Unnamed: 0.1,ID,State,City,agyaddr,xobsyr_0,Illicit_Days5,Illicit_Cens5,female_cd,nonwhite_cd,unemplmt_cd,prsatx_cd,gvsg_cd,CWSg_0_cd,srprobg_cd,dssg_0_cd,epsg_0_cd,adhdg_0_cd,cdsg_0_cd,cjsig_0_cd,lrig_0_cd,srig_0_cd,SESg_0_cd,r4ag_0_cd,nonillicit_flag,primsev_cd_1,primsev_cd_2,primsev_cd_3,primsev_cd_4,primsev_cd_5,primsev_cd_6,B2a_0g,SUDSy_0,SDSl_0,Address,lat,lng,state_name,county_FIPS,block_FIPS
0,1104,1105,2799,CA,San Diego,3274 Rosecrans Street,2003,175,0,1,0,0,1,2,0,1,1,1,2,0,2,0,0,1,1,0,0,1,0,0,0,0,2,4,7,"3274 Rosecrans Street, San Diego, CA",32.750859,-117.205031,California,6073.0,6.073007e+13
1,11342,11349,14812,AZ,Tucson,3130 E. Broadway Blvd. Suite 180,2007,365,0,0,1,0,1,2,0,2,1,1,1,1,2,1,2,0,2,1,1,0,0,0,0,0,1,5,3,"3130 E. Broadway Blvd. Suite 180, Tucson, AZ",32.220280,-110.963329,Arizona,4019.0,4.019001e+13
2,1686,1688,18928,TN,Knoxville,1534 White Ave,2011,365,0,0,0,1,1,2,0,2,2,2,1,1,2,1,1,0,2,1,0,0,1,0,0,0,1,10,7,"1534 White Ave, Knoxville, TN",35.958631,-83.929508,Tennessee,47093.0,4.709300e+14
3,13838,13848,17706,AZ,Tucson,"3130 E. Broadway Blvd, Suite 180",2009,358,0,0,1,0,0,2,0,1,1,1,1,2,0,2,1,0,0,1,0,0,1,0,0,0,1,7,5,"3130 E. Broadway Blvd, Suite 180, Tucson, AZ",32.220280,-110.963329,Arizona,4019.0,4.019001e+13
4,4234,4238,6981,PA,Newtown,P.O. Box 908,2005,365,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,1,0,1,0,0,1,0,0,0,1,2,1,"P.O. Box 908, Newtown, PA",40.216484,-74.944974,Pennsylvania,42017.0,4.201711e+14


In [26]:
df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True)
df.shape

(21264, 39)

In [27]:
# start with raw numbers, end with FIPS code and concentrated disadvantage feature
def calculate_concentrated_disadvantage(filename):
    df_census = pd.read_csv(filename, encoding = "ISO-8859-1")
    
    cols_to_use = ['SE_A01001_001','SE_A01001_002','SE_A01001_003','SE_A01001_004','SE_A01001_005','SE_A10008_001',
                   'SE_A10008_006','SE_A10008_009','SE_A17005_001','SE_A17005_003','SE_A10014_001','SE_A10014_002',
                   'SE_A13003A_001','SE_A13003A_002','SE_A13003B_001','SE_A13003B_002','SE_A13003C_001',
                   'SE_A13003C_002','Geo_FIPS']
    df_census = df_census[cols_to_use]

    # calculate percentage under 18
    age_cols = ['SE_A01001_002', 'SE_A01001_003', 'SE_A01001_004', 'SE_A01001_005']
    df_census['%_U18'] = df_census[age_cols].sum(axis=1) / df_census['SE_A01001_001']
    age_cols.append('SE_A01001_001')
    df_census.drop(columns=age_cols, inplace=True)

    # calculate percentage female households
    female_householder_cols = ['SE_A10008_006', 'SE_A10008_009']
    df_census['%_female_household'] = df_census[female_householder_cols].sum(axis=1) / df_census['SE_A10008_001']
    female_householder_cols.append('SE_A10008_001')
    df_census.drop(columns=female_householder_cols, inplace=True)

    # calculate percentage unemployed
    df_census['%_unemployed'] = df_census['SE_A17005_003'] / df_census['SE_A17005_001']
    df_census.drop(columns=['SE_A17005_003', 'SE_A17005_001'], inplace=True)

    # calculate percentage households with public assistance income 
    df_census['%_public_assistance'] = df_census['SE_A10014_002'] / df_census['SE_A10014_001']
    df_census.drop(columns=['SE_A10014_001', 'SE_A10014_002'], inplace=True)

    # calculate percentage living in poverty
    population_cols = ['SE_A13003A_001', 'SE_A13003B_001', 'SE_A13003C_001']
    poverty_cols = ['SE_A13003A_002', 'SE_A13003B_002', 'SE_A13003C_002']
    df_census['%_poverty'] = df_census[poverty_cols].sum(axis=1) / df_census[population_cols].sum(axis=1)
    df_census.drop(columns=(population_cols + poverty_cols), inplace=True)

    """# calculate z-scores of 5 census variables
    cols_to_z_score = ['%_U18', '%_female_household', '%_unemployed', '%_public_assistance', '%_poverty']
    for col in cols_to_z_score:
        df_census[col] = (df_census[col] - df_census[col].mean())/df_census[col].std(ddof=0)

    # average z-scored census variables to get concentrated disadvantage
    df_census['concentrated_disadvantage'] = df_census[cols_to_z_score].mean(axis=1)
    
    cuttoff = df_census['concentrated_disadvantage'].quantile(.75)
    df_census['hcd'] = np.where(df_census['concentrated_disadvantage']>=cuttoff, 1, 0)
    df_census.drop(columns='concentrated_disadvantage', inplace=True)"""
    
    df_census['xobsyr_0'] = year
    return df_census

In [28]:
%%time

from functools import reduce

years = [2012, 2011, 2010, 2009, 2008, 2007, 2006] # NOTE: must be in descending order
frames = []

# collect at county level
for year in years:
    df_census = calculate_concentrated_disadvantage('census_data/county/ACS1yr' + str(year) + 'Census.csv')
    df_census['gran'] = 0
    frames.append(df_census)

# collect at census_tract level
for year in years[:-3]:
    df_census = calculate_concentrated_disadvantage('census_data/census_tract/ACS5yr' + str(year) + 'Census.csv')
    df_census['gran'] = 1
    frames.append(df_census)    
    
hcd = pd.concat(frames)
hcd.head()

CPU times: user 2.96 s, sys: 607 ms, total: 3.57 s
Wall time: 3.63 s


,Geo_FIPS,%_U18,%_female_household,%_unemployed,%_public_assistance,%_poverty,xobsyr_0,gran
0,1003,0.222837,0.263844,0.087921,0.010490,0.140557,2012,0
1,1015,0.224347,0.310551,0.153805,0.023790,0.221665,2012,0
2,1043,0.223309,0.294499,0.075186,0.024197,0.197444,2012,0
3,1049,0.253700,0.238181,0.081434,0.017791,0.132595,2012,0
4,1051,0.230475,0.280535,0.061826,0.012887,0.129935,2012,0


In [29]:
hcd.tail()

,Geo_FIPS,%_U18,%_female_household,%_unemployed,%_public_assistance,%_poverty,xobsyr_0,gran
66317,72153750501,0.281832,0.407902,0.236864,0.048585,0.543634,2009,1
66318,72153750502,0.241569,0.437783,0.290934,0.049774,0.496850,2009,1
66319,72153750503,0.254597,0.402473,0.230878,0.045330,0.677982,2009,1
66320,72153750601,0.282056,0.378886,0.215434,0.010557,0.436934,2009,1
66321,72153750602,0.236600,0.396207,0.282520,0.100105,0.664333,2009,1


In [30]:
# group raw variables 

raw_census_vars = ['%_U18', '%_female_household', '%_unemployed', '%_public_assistance', '%_poverty']

for feat in raw_census_vars:
    low = hcd[feat].min()
    high = hcd[feat].max()
    quarter = (high - low) / 4

    new_feat = feat + 'g'
    
    # separate into bottom 25%, middle 50%, and upper 25%
    hcd[new_feat] = pd.cut(hcd[feat], [low,low+quarter,low+3*quarter,high], labels=[0, 1, 2])
    hcd.drop(columns=feat, inplace=True)

hcd.tail()

,Geo_FIPS,xobsyr_0,gran,%_U18g,%_female_householdg,%_unemployedg,%_public_assistanceg,%_povertyg
66317,72153750501,2009,1,1,1,0,0,1
66318,72153750502,2009,1,0,1,1,0,1
66319,72153750503,2009,1,1,1,0,0,1
66320,72153750601,2009,1,1,1,0,0,1
66321,72153750602,2009,1,0,1,1,0,1


In [31]:
hcd['Geo_FIPS'] = hcd['Geo_FIPS'].astype(str)
hcd.dtypes

Geo_FIPS                  object
xobsyr_0                   int64
gran                       int64
%_U18g                  category
%_female_householdg     category
%_unemployedg           category
%_public_assistanceg    category
%_povertyg              category
dtype: object

In [32]:
# pull out unique county/year combinations
fips_df = df[['block_FIPS', 'county_FIPS', 'xobsyr_0', 'Address']]
fips_df = fips_df.drop_duplicates(subset=['Address', 'xobsyr_0'])
fips_df['block_FIPS'] = fips_df['block_FIPS'].astype(str).str[:-2] # 54.099e15 -> 540990051001055.0 -> 540990051001055
fips_df['county_FIPS'] = fips_df['county_FIPS'].astype(str).str[:-2] # 54099.0 -> 54099
fips_df.head()

,block_FIPS,county_FIPS,xobsyr_0,Address
0,60730065001038,6073,2003,"3274 Rosecrans Street, San Diego, CA"
1,40190008002011,4019,2007,"3130 E. Broadway Blvd. Suite 180, Tucson, AZ"
2,470930009011002,47093,2011,"1534 White Ave, Knoxville, TN"
3,40190008002011,4019,2009,"3130 E. Broadway Blvd, Suite 180, Tucson, AZ"
4,420171052023041,42017,2005,"P.O. Box 908, Newtown, PA"


In [33]:
fips_df.shape

(596, 4)

In [34]:
from tqdm.notebook import tqdm # used to show progress bar

for i,row in tqdm(fips_df.iterrows(), total=fips_df.shape[0]):
    if len(row['block_FIPS'].strip()) != 15:
        fips_df.set_value(i, 'block_FIPS', '0' + row['block_FIPS'].strip())
    if len(row['county_FIPS'].strip()) != 5:
        fips_df.set_value(i, 'county_FIPS', '0' + row['county_FIPS'].strip())
        
fips_df = fips_df.astype(str)
fips_df.head()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys


,block_FIPS,county_FIPS,xobsyr_0,Address
0,060730065001038,06073,2003,"3274 Rosecrans Street, San Diego, CA"
1,040190008002011,04019,2007,"3130 E. Broadway Blvd. Suite 180, Tucson, AZ"
2,470930009011002,47093,2011,"1534 White Ave, Knoxville, TN"
3,040190008002011,04019,2009,"3130 E. Broadway Blvd, Suite 180, Tucson, AZ"
4,420171052023041,42017,2005,"P.O. Box 908, Newtown, PA"


In [35]:
import numpy as np

fips_df = fips_df[fips_df.xobsyr_0 >= '2006']

fips_df['gran'] = np.nan
fips_df['%_U18g'] = np.nan
fips_df['%_female_householdg'] = np.nan
fips_df['%_unemployedg'] = np.nan
fips_df['%_public_assistanceg'] = np.nan
fips_df['%_povertyg'] = np.nan

fips_df.shape

(474, 10)

In [36]:
# merge with patient location data

print('Merging at census tract level...')
for i,row in tqdm(fips_df.iterrows(), total=fips_df.shape[0]):
    try:
        block_FIPS = row['block_FIPS']
        xobsyr_0 = row['xobsyr_0']
        hcd_df = hcd[hcd.xobsyr_0 == int(xobsyr_0)]
        hcd_df = hcd_df[hcd_df.gran == 1]
        for j,r in hcd_df.iterrows():
            #print(r['xobsyr_0'], '=>', xobsyr_0, block_FIPS, '=>', r['Geo_FIPS'])
            if block_FIPS.startswith(r['Geo_FIPS']): 
                #print('here!!')
                fips_df.set_value(i, 'gran', 1)
                fips_df.set_value(i, '%_U18g', r['%_U18g'])
                fips_df.set_value(i, '%_female_householdg', r['%_female_householdg'])
                fips_df.set_value(i, '%_unemployedg', r['%_unemployedg'])
                fips_df.set_value(i, '%_public_assistanceg', r['%_public_assistanceg'])
                fips_df.set_value(i, '%_povertyg', r['%_povertyg'])
                break
    except Exception as e:
        print('Error with line ' + i, e)
            
fips_df.head()

Merging at census tract level...


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] o

,block_FIPS,county_FIPS,xobsyr_0,Address,gran,%_U18g,%_female_householdg,%_unemployedg,%_public_assistanceg,%_povertyg
1,040190008002011,04019,2007,"3130 E. Broadway Blvd. Suite 180, Tucson, AZ",NaN,NaN,NaN,NaN,NaN,NaN
2,470930009011002,47093,2011,"1534 White Ave, Knoxville, TN",1.0,0.0,2.0,0.0,NaN,2.0
3,040190008002011,04019,2009,"3130 E. Broadway Blvd, Suite 180, Tucson, AZ",NaN,NaN,NaN,NaN,NaN,NaN
6,483396931021019,48339,2008,"Campus Box 2447, Huntsville, TX",NaN,NaN,NaN,NaN,NaN,NaN
7,484530023083000,48453,2008,"2515 S. Congress Ave., Austin, TX",NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
fips_df.shape[0] - fips_df['gran'].isna().sum() # see how many rows have census data

196

In [46]:
print('Merging remaining with county level data...')
for i,row in tqdm(fips_df.iterrows(), total=fips_df.shape[0]):
    try:
        if row['gran'] != 1: # if not filled by block_FIPS
            print('here!', row['gran'])
            county_FIPS = row['county_FIPS']
            xobsyr_0 = row['xobsyr_0']
            hcd_df = hcd[hcd.xobsyr_0 == int(xobsyr_0)]
            hcd_df = hcd_df[hcd_df.gran == 0]
            for j,r in hcd_df.iterrows():
                #print(county_FIPS, '=>', r['Geo_FIPS'])
                try:
                    if int(r['Geo_FIPS']) == int(county_FIPS): 
                        #print('here!!', r['Geo_FIPS'], '==', county_FIPS)
                        fips_df.set_value(i, 'gran', 0)
                        fips_df.set_value(i, '%_U18g', r['%_U18g'])
                        fips_df.set_value(i, '%_female_householdg', r['%_female_householdg'])
                        fips_df.set_value(i, '%_unemployedg', r['%_unemployedg'])
                        fips_df.set_value(i, '%_public_assistanceg', r['%_public_assistanceg'])
                        fips_df.set_value(i, '%_povertyg', r['%_povertyg'])
                        break
                except Exception as e:
                    print('Error comparing', r['Geo_FIPS'], 'with', county_FIPS)
    except Exception as e:
        print('Error with line ' + i, e)

Merging remaining with county level data...


here! 0.0
here!! 4019 == 04019
here! 0.0
here!! 4019 == 04019
here! 0.0
here!! 48339 == 48339
here! 0.0


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .

here!! 48453 == 48453
here! 0.0
here!! 1015 == 01015
here! 0.0
here!! 48215 == 48215
here! 0.0
here!! 39153 == 39153
here! 0.0
here!! 29077 == 29077
here! 0.0
here!! 6037 == 06037
here! 0.0
here!! 6019 == 06019
here! 0.0
here!! 12103 == 12103
here! 0.0
here!! 26163 == 26163
here! 0.0
here!! 8031 == 08031
here! 0.0
here!! 6037 == 06037
here! 0.0
here!! 42101 == 42101
here! 0.0
here!! 48029 == 48029
here! 0.0
here!! 4019 == 04019
here! 0.0
here!! 6037 == 06037
here! 0.0
here!! 12105 == 12105
here! 0.0
here!! 48479 == 48479
here! 0.0
here!! 6037 == 06037
here! 0.0
here!! 48141 == 48141
here! 0.0
here!! 6037 == 06037
here! 0.0
here!! 12009 == 12009
here! 0.0
here!! 25027 == 25027
here! 0.0
here!! 34025 == 34025
here! 0.0
here!! 8031 == 08031
here! 0.0
here!! 8031 == 08031
here! 0.0
here!! 48479 == 48479
here! 0.0
here!! 12103 == 12103
here! 0.0
here!! 6111 == 06111
here! 0.0
here!! 12086 == 12086
here! 0.0
here!! 4019 == 04019
here! 0.0
here!! 6083 == 06083
here! 0.0
here!! 6073 == 06073
h

Error comparing 33009 with 0n
Error comparing 33011 with 0n
Error comparing 33013 with 0n
Error comparing 33015 with 0n
Error comparing 33017 with 0n
Error comparing 34001 with 0n
Error comparing 34003 with 0n
Error comparing 34005 with 0n
Error comparing 34007 with 0n
Error comparing 34009 with 0n
Error comparing 34011 with 0n
Error comparing 34013 with 0n
Error comparing 34015 with 0n
Error comparing 34017 with 0n
Error comparing 34019 with 0n
Error comparing 34021 with 0n
Error comparing 34023 with 0n
Error comparing 34025 with 0n
Error comparing 34027 with 0n
Error comparing 34029 with 0n
Error comparing 34031 with 0n
Error comparing 34033 with 0n
Error comparing 34035 with 0n
Error comparing 34037 with 0n
Error comparing 34039 with 0n
Error comparing 34041 with 0n
Error comparing 35001 with 0n
Error comparing 35005 with 0n
Error comparing 35013 with 0n
Error comparing 35025 with 0n
Error comparing 35031 with 0n
Error comparing 35035 with 0n
Error comparing 35043 with 0n
Error comp

...

Error comparing 18035 with 0n
Error comparing 18039 with 0n
Error comparing 18043 with 0n
Error comparing 18053 with 0n
Error comparing 18057 with 0n
Error comparing 18059 with 0n
Error comparing 18063 with 0n
Error comparing 18067 with 0n
Error comparing 18081 with 0n
Error comparing 18085 with 0n
Error comparing 18089 with 0n
Error comparing 18091 with 0n
Error comparing 18095 with 0n
Error comparing 18097 with 0n
Error comparing 18105 with 0n
Error comparing 18109 with 0n
Error comparing 18127 with 0n
Error comparing 18141 with 0n
Error comparing 18157 with 0n
Error comparing 18163 with 0n
Error comparing 18167 with 0n
Error comparing 18177 with 0n
Error comparing 19013 with 0n
Error comparing 19049 with 0n
Error comparing 19061 with 0n
Error comparing 19103 with 0n
Error comparing 19113 with 0n
Error comparing 19153 with 0n
Error comparing 19155 with 0n
Error comparing 19163 with 0n
Error comparing 19169 with 0n
Error comparing 19193 with 0n
Error comparing 20015 with 0n
Error comp

Error comparing 39169 with 0n
Error comparing 39173 with 0n
Error comparing 40017 with 0n
Error comparing 40027 with 0n
Error comparing 40031 with 0n
Error comparing 40037 with 0n
Error comparing 40101 with 0n
Error comparing 40109 with 0n
Error comparing 40119 with 0n
Error comparing 40125 with 0n
Error comparing 40131 with 0n
Error comparing 40143 with 0n
Error comparing 40145 with 0n
Error comparing 41003 with 0n
Error comparing 41005 with 0n
Error comparing 41017 with 0n
Error comparing 41019 with 0n
Error comparing 41029 with 0n
Error comparing 41033 with 0n
Error comparing 41035 with 0n
Error comparing 41039 with 0n
Error comparing 41043 with 0n
Error comparing 41047 with 0n
Error comparing 41051 with 0n
Error comparing 41053 with 0n
Error comparing 41059 with 0n
Error comparing 41067 with 0n
Error comparing 41071 with 0n
Error comparing 42001 with 0n
Error comparing 42003 with 0n
Error comparing 42005 with 0n
Error comparing 42007 with 0n
Error comparing 42011 with 0n
Error comp

here!! 48215 == 48215
here! 0.0
here!! 9001 == 09001
here! 0.0
here!! 51680 == 51680
here! 0.0
here!! 9001 == 09001
here! 0.0
here!! 8001 == 08001
here! 0.0
here!! 42017 == 42017
here! 0.0
here!! 42017 == 42017
here! 0.0
here!! 50007 == 50007
here! 0.0
here!! 44007 == 44007
here! 0.0
here!! 48201 == 48201
here! 0.0
here!! 4013 == 04013
here! 0.0
here!! 25009 == 25009
here! 0.0
here!! 9009 == 09009
here! 0.0
here!! 4019 == 04019
here! 0.0
here!! 6071 == 06071
here! 0.0
here!! 25017 == 25017
here! nan
here! 0.0
here!! 4013 == 04013
here! nan
here! 0.0
here!! 29510 == 29510
here! 0.0
here!! 4019 == 04019
here! 0.0
here!! 17113 == 17113
here! 0.0
here!! 4019 == 04019
here! 0.0
here!! 6037 == 06037
here! 0.0
here!! 39035 == 39035
here! 0.0
here!! 48453 == 48453
here! 0.0
here!! 33011 == 33011
here! 0.0
here!! 25027 == 25027
here! nan
here! 0.0
here!! 25017 == 25017
here! 0.0
here!! 17113 == 17113
here! 0.0
here!! 6111 == 06111
here! 0.0
here!! 48479 == 48479
here! 0.0
here!! 12103 == 12103


Error comparing 23019 with 0n
Error comparing 23031 with 0n
Error comparing 24001 with 0n
Error comparing 24003 with 0n
Error comparing 24005 with 0n
Error comparing 24009 with 0n
Error comparing 24013 with 0n
Error comparing 24015 with 0n
Error comparing 24017 with 0n
Error comparing 24021 with 0n
Error comparing 24025 with 0n
Error comparing 24027 with 0n
Error comparing 24031 with 0n
Error comparing 24033 with 0n
Error comparing 24037 with 0n
Error comparing 24043 with 0n
Error comparing 24045 with 0n
Error comparing 24510 with 0n
Error comparing 25001 with 0n
Error comparing 25003 with 0n
Error comparing 25005 with 0n
Error comparing 25009 with 0n
Error comparing 25011 with 0n
Error comparing 25013 with 0n
Error comparing 25015 with 0n
Error comparing 25017 with 0n
Error comparing 25021 with 0n
Error comparing 25023 with 0n
Error comparing 25025 with 0n
Error comparing 25027 with 0n
Error comparing 26005 with 0n
Error comparing 26017 with 0n
Error comparing 26021 with 0n
Error comp

Error comparing 42129 with 0n
Error comparing 42133 with 0n
Error comparing 44003 with 0n
Error comparing 44005 with 0n
Error comparing 44007 with 0n
Error comparing 44009 with 0n
Error comparing 45003 with 0n
Error comparing 45007 with 0n
Error comparing 45013 with 0n
Error comparing 45015 with 0n
Error comparing 45019 with 0n
Error comparing 45031 with 0n
Error comparing 45035 with 0n
Error comparing 45041 with 0n
Error comparing 45045 with 0n
Error comparing 45047 with 0n
Error comparing 45051 with 0n
Error comparing 45057 with 0n
Error comparing 45059 with 0n
Error comparing 45063 with 0n
Error comparing 45073 with 0n
Error comparing 45075 with 0n
Error comparing 45077 with 0n
Error comparing 45079 with 0n
Error comparing 45083 with 0n
Error comparing 45085 with 0n
Error comparing 45091 with 0n
Error comparing 46099 with 0n
Error comparing 46103 with 0n
Error comparing 47001 with 0n
Error comparing 47009 with 0n
Error comparing 47011 with 0n
Error comparing 47037 with 0n
Error comp

In [39]:
#fips_df.shape[0] - fips_df['gran'].isna().sum() # see how many addresses have census data

In [40]:
fips_df.head()

,block_FIPS,county_FIPS,xobsyr_0,Address,gran,%_U18g,%_female_householdg,%_unemployedg,%_public_assistanceg,%_povertyg
1,040190008002011,04019,2007,"3130 E. Broadway Blvd. Suite 180, Tucson, AZ",0.0,0.0,1.0,0.0,0.0,0.0
2,470930009011002,47093,2011,"1534 White Ave, Knoxville, TN",1.0,0.0,2.0,0.0,NaN,2.0
3,040190008002011,04019,2009,"3130 E. Broadway Blvd, Suite 180, Tucson, AZ",0.0,0.0,1.0,0.0,0.0,0.0
6,483396931021019,48339,2008,"Campus Box 2447, Huntsville, TX",0.0,1.0,0.0,0.0,0.0,0.0
7,484530023083000,48453,2008,"2515 S. Congress Ave., Austin, TX",0.0,1.0,1.0,0.0,0.0,0.0


In [41]:
fips_df['xobsyr_0'] = fips_df['xobsyr_0'].astype(int)

In [42]:
df = df.merge(fips_df, on=['Address', 'xobsyr_0'], how='left')
df.head(50)

,ID,State,City,agyaddr,xobsyr_0,Illicit_Days5,Illicit_Cens5,female_cd,nonwhite_cd,unemplmt_cd,prsatx_cd,gvsg_cd,CWSg_0_cd,srprobg_cd,dssg_0_cd,epsg_0_cd,adhdg_0_cd,cdsg_0_cd,cjsig_0_cd,lrig_0_cd,srig_0_cd,SESg_0_cd,r4ag_0_cd,nonillicit_flag,primsev_cd_1,primsev_cd_2,primsev_cd_3,primsev_cd_4,primsev_cd_5,primsev_cd_6,B2a_0g,SUDSy_0,SDSl_0,Address,lat,lng,state_name,county_FIPS_x,block_FIPS_x,block_FIPS_y,county_FIPS_y,gran,%_U18g,%_female_householdg,%_unemployedg,%_public_assistanceg,%_povertyg
0,2799,CA,San Diego,3274 Rosecrans Street,2003,175,0,1,0,0,1,2,0,1,1,1,2,0,2,0,0,1,1,0,0,1,0,0,0,0,2,4,7,"3274 Rosecrans Street, San Diego, CA",32.750859,-117.205031,California,6073.0,6.073007e+13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14812,AZ,Tucson,3130 E. Broadway Blvd. Suite 180,2007,365,0,0,1,0,1,2,0,2,1,1,1,1,2,1,2,0,2,1,1,0,0,0,0,0,1,5,3,"3130 E. Broadway Blvd. Suite 180, Tucson, AZ",32.220280,-110.963329,Arizona,4019.0,4.019001e+13,040190008002011,04019,0.0,0.0,1.0,0.0,0.0,0.0
2,18928,TN,Knoxville,1534 White Ave,2011,365,0,0,0,1,1,2,0,2,2,2,1,1,2,1,1,0,2,1,0,0,1,0,0,0,1,10,7,"1534 White Ave, Knoxville, TN",35.958631,-83.929508,Tennessee,47093.0,4.709300e+14,470930009011002,47093,1.0,0.0,2.0,0.0,NaN,2.0
3,17706,AZ,Tucson,"3130 E. Broadway Blvd, Suite 180",2009,358,0,0,1,0,0,2,0,1,1,1,1,2,0,2,1,0,0,1,0,0,1,0,0,0,1,7,5,"3130 E. Broadway Blvd, Suite 180, Tucson, AZ",32.220280,-110.963329,Arizona,4019.0,4.019001e+13,040190008002011,04019,0.0,0.0,1.0,0.0,0.0,0.0
4,6981,PA,Newtown,P.O. Box 908,2005,365,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,1,0,1,0,0,1,0,0,0,1,2,1,"P.O. Box 908, Newtown, PA",40.216484,-74.944974,Pennsylvania,42017.0,4.201711e+14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2700,IA,Des Moines,1111 University Avenue,2005,100,0,1,1,0,1,2,0,2,2,1,2,1,0,2,1,0,1,0,0,1,0,0,0,0,1,9,7,"1111 University Avenue, Des Moines, IA",41.600704,-93.632392,Iowa,19153.0,1.915301e+14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,15446,TX,Huntsville,Campus Box 2447,2008,365,0,0,1,0,0,2,0,1,1,1,2,1,1,2,1,0,2,1,1,0,0,0,0,0,1,5,2,"Campus Box 2447, Huntsville, TX",30.291236,-95.405465,Texas,48339.0,4.833969e+14,483396931021019,48339,0.0,1.0,0.0,0.0,0.0,0.0
7,12304,TX,Austin,2515 S. Congress Ave.,2008,30,1,1,1,0,1,0,0,1,0,1,1,1,2,1,1,0,2,1,0,0,1,0,0,0,0,7,4,"2515 S. Congress Ave., Austin, TX",30.236851,-97.754927,Texas,48453.0,4.845300e+14,484530023083000,48453,0.0,1.0,1.0,0.0,0.0,0.0
8,7338,CA,Daly City,7100 Mission Street,2005,364,0,1,0,0,0,0,0,0,2,1,1,1,0,1,2,1,2,1,1,0,0,0,0,0,1,3,1,"7100 Mission Street, Daly City, CA",37.708340,-122.454066,California,6075.0,6.075026e+13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,21801,AL,Anniston,P.O. Box 1347,2011,183,0,1,0,1,1,0,0,2,1,1,0,0,1,1,1,0,2,0,0,0,0,0,1,0,2,11,7,"P.O. Box 1347, Anniston, AL",33.629397,-85.829099,Alabama,1015.0,1.015001e+13,010150008001217,01015,0.0,0.0,1.0,0.0,0.0,0.0


In [43]:
df.shape[0] - df['gran'].isna().sum() # see how many patients have census data

15376

In [44]:
df.isna().sum()

ID                         0
State                      0
City                       0
agyaddr                    0
xobsyr_0                   0
Illicit_Days5              0
Illicit_Cens5              0
female_cd                  0
nonwhite_cd                0
unemplmt_cd                0
prsatx_cd                  0
gvsg_cd                    0
CWSg_0_cd                  0
srprobg_cd                 0
dssg_0_cd                  0
epsg_0_cd                  0
adhdg_0_cd                 0
cdsg_0_cd                  0
cjsig_0_cd                 0
lrig_0_cd                  0
srig_0_cd                  0
SESg_0_cd                  0
r4ag_0_cd                  0
nonillicit_flag            0
primsev_cd_1               0
primsev_cd_2               0
primsev_cd_3               0
primsev_cd_4               0
primsev_cd_5               0
primsev_cd_6               0
B2a_0g                     0
SUDSy_0                    0
SDSl_0                     0
Address                    0
lat           